# Glacier advance and retreat

Goals of this notebook:

- Understand the concept of the equilibrium line altitude (ELA)
- Understand the influence of glacier mass balance on the ELA
- Be able to explain glacier advance and retreat in response to a change in the ELA

In [ ]:
# Constants
from oggm import cfg
cfg.initialize_minimal()

# OGGM flowline model
from oggm.core.flowline import RectangularBedFlowline

# Scientific packages
import numpy as np

# OGGM mass-balance model
from oggm.core.massbalance import LinearMassBalance

# There are several numerical implementations in OGGM core. We use the "FluxBasedModel"
from oggm.core.flowline import FluxBasedModel as FlowlineModel

# Plot glacier graphics edu function
from oggm_edu import plot_glacier_graphics

In [ ]:
# Plotting libraries and plot style
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_context('notebook')
sns.set_style('ticks')

## A simulation of glacier advance and retreat with OGGM

We start by defining a idealised glacier, then we let the glacier grow until it reaches its equilibrium state. Then we can simulate glacier advance and retreat. We represent the different steps illustrated in the [open glacier graphics from the OGGM-EDU website](https://edu.oggm.org/en/latest/glacier_basics.html) (made by Anne Maussion, [Atelier les Gros yeux](http://atelierlesgrosyeux.com/)).

### Initialisation: Let's define our glacier

First, as always, we define a linear bedrock profile:

In [ ]:
# define horizontal resolution of the model:
# nx: number of grid points
# map_dx: grid point spacing in meters
nx = 200
map_dx = 200

# define glacier top and bottom altitudes in meters
top = 5000 
bottom = 0

# create a linear bedrock profile from top to bottom
bed_h = np.linspace(top, bottom, nx)

# calculate the distance from the top to the bottom of the glacier in km
distance_along_glacier = np.linspace(0, nx, nx) * map_dx * 1e-3

Often glaciers are wider in the accumulation area, so we construct our glacier such that it is wider above the ELA. 

In [ ]:
# glacier width at the top of the accumulation area: m
ACCW = 1000

# glacier width at the equilibrium line altitude: m
ELAW = 500

# fraction of vertical grid points occupied by accumulation area
NZ = 1 / 3

# accumulation area occupies a fraction NZ of the total glacier extent
acc_width = np.linspace(ACCW, ELAW, int(nx * NZ))

# ablation area occupies a fraction 1 - NZ of the total glacier extent
abl_width = np.tile(ELAW, nx - len(acc_width))

# glacier widths profile
widths = np.hstack([acc_width, abl_width])

In [ ]:
# Plot glacier geometry
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(10, 9),
                               gridspec_kw={'height_ratios': [2, 1]},
                               sharex=True)
# Plot the bed.
ax1.plot(distance_along_glacier, bed_h, ls='--', c='k')
ax1.set_ylabel('Altitude (m)')
ax1.set_ylim(0, 5200)
# Plot the width to the distance along glacier.
ax2.fill_between(distance_along_glacier, -widths/2, widths/2,
                 edgecolor='tab:blue', facecolor='white')
ax2.axhline(0, c='k', lw=0.4)
ax2.set_ylabel('Glacier width across glacier (m)')
ax2.set_xlabel('Distance along glacier  (km)');
ax2.set_xlim(0, 45);
ax1.set_title('Slope and width-elevation profile of the bedrock');

**Now we have defined the idealised width-elevation profile of our glacier.** The next step is to define the shape of our cross-section. OGGM supports three different shapes (rectangular, parabolic and trapezoidal); for our experiment we go with the rectangular one:

In [ ]:
# describe the widths in "grid points" for the model, based on grid point spacing map_dx
mwidths = np.zeros(nx) + widths / map_dx

# define the glacier flowline
initial_flowline = RectangularBedFlowline(surface_h=bed_h,
                                          bed_h=bed_h,
                                          widths=mwidths,
                                          map_dx=map_dx)

Now we need to define the mass balance distribution over the glacier, after which the glacier can grow to its initial equilibrium state.

#### **Mass balance and the equilibrium line altitude**

The **mass balance** is the result of several processes that either add mass to the glacier (**accumulation**) or remove mass from the glacier (**ablation**). You can find more details in the [accumulation and ablation notebook](accumulation_and_ablation.ipynb). As a summary the following glacier graphics illustrate this relationship: 

- The left graphic represents a theoretical mass accumulation over the whole glacier depicted by the blue ice volume on top of the grey glacier body.
- In the graphic in the middle a theoretical mass ablation is marked as red ice volume.
- The  graphic on the right shows the resulting mass balance with blue and red arrows in combination with the blue and red volume changes on top of the grey glacier body. In the central part of the glacier where the red line lies directly on the grey glacier body ablation and accumulation canceled each other out.

<img src="https://raw.githubusercontent.com/OGGM/glacier-graphics/master/glacier_intro/png/glacier_03.png" width="33%" align="left">
<img src="https://raw.githubusercontent.com/OGGM/glacier-graphics/master/glacier_intro/png/glacier_04.png" width="33%" align="left">
<img src="https://raw.githubusercontent.com/OGGM/glacier-graphics/master/glacier_intro/png/glacier_05.png" width="33%" align="left">

*Source: [open glacier graphics](http://edu.oggm.org/en/latest/glacier_basics.html) on OGGM-Edu, made by Anne Maussion ([Atelier les Gros yeux](http://atelierlesgrosyeux.com/))*.

The rates of accumulation and ablation processes, summed over the glacier and over time, determine the *glacier mass balance*: $\dot{m}$, the change in total mass of snow and ice,

$$\dot{m} = \text{accumulation} + \text{ablation}.$$

Since accumulation and ablation generally vary with height, also the glacier mass balance is a function of elevation,

$$\dot{m}(z) = \text{accumulation}(z) + \text{ablation}(z).$$

Mass is continuously redistributed in a glacier: accumulated mass at the top of the glacier is transported downglacier. The driving force of this *ice flow* is gravity. Thus, the mass balance of a region on a glacier depends not only on the mass exchanges induced by accumulation and ablation, but also on the gravity driven transport of ice from the acccumulation area to the ablation area. The *ice flow* is indicated by the grey arrow in this figure:

<img src="https://raw.githubusercontent.com/OGGM/glacier-graphics/master/glacier_intro/png/glacier_06.png" width="60%">

The altitude where $\dot{m}(z) = 0$ is called the *equilibrium line altitude*, short ELA. Hence, the ELA is the altitude where accumulation processes and ablation processes balance each other - in theory. However, in reality the ELA does not exactly exist and can only be approximated.

<img src="https://raw.githubusercontent.com/OGGM/glacier-graphics/master/glacier_intro/png/glacier_07.png" width="60%">

**We want to reproduce these processes above in an experiment using OGGM**.

For this purpose we start with defining a linear mass balance model of the form

$$\dot{m}(z) = (z - ELA) \frac{d\dot{m}}{dz},$$

with the mass balance gradient $\frac{d\dot{m}}{dz}$:

In [ ]:
# mass balance gradient with respect to elevation in mm w.e. m^-1 yr^-1
mb_grad = 7

Define the ELA to be as close as possible to the ELA width. With this we can define the mass balance model.

In [ ]:
# equilibrium line altitude: height where the glacier width first hits ELAW
ela = bed_h[np.where(widths==ELAW)[0][0]]
print('ELA: {:.2f} m'.format(ela))

# defining the mass balance model
mb_model = LinearMassBalance(ela, grad=mb_grad)

Now that we have all the ingredients to run the model, we just have to initialise it:

In [ ]:
# The model requires the initial glacier bed, a mass balance model, and an initial time (the year y0)
model = FlowlineModel(initial_flowline, mb_model=mb_model, y0=0.,
                      min_dt=0, cfl_number=0.01)

### Glacier in Equilibrium

For a glacier to be in equilibrium, we require the specific mass balance (accumulation + ablation) to be zero averaged over a year on the glacier. A glacier is in equilibrium if the glacier will neither retreat nor advance from one year to the next year if the climate stays constant, i.e. if we don't change the ELA. 

Let's run the model until our test glacier is in equilibrium:

In [ ]:
model.run_until_equilibrium()

Now, we want to calculate the net mass balance (= glacier mass distribution without considering ice flow). First we calculate the accumulation over the glacier as a function of altitude:

In [ ]:
# get the equilibrium surface height of our test glacier in equilibrium
current_sfc = model.fls[0].surface_h

In [ ]:
# First we find the terminus of the glacier, where the glacier surface is
# intersecting the bed.
terminus = current_sfc[current_sfc <= bed_h][0]

# We then need the surface closest to the ELA
ela_sfc = current_sfc[np.abs(current_sfc - ela).argmin()]

# First we calculate the accumulation at the ELA. This is calculated so that
# the accumulation at terminus is 0.
ela_accumulation = (- mb_grad * (terminus - ela_sfc) * 
                    mwidths[current_sfc == terminus] * 1e-3)

# We can now calculate the annual accumulation as a function of the altitude.
# Scaled by the width of the glacier.
accumulation = (ela_accumulation + mb_grad *
                (current_sfc[current_sfc >= terminus] - ela_sfc) *
                1e-3 * mwidths[current_sfc >= terminus])

# Append the zero accumulation downstream of glacier terminus
accumulation = np.hstack([accumulation,
                          np.zeros(len(current_sfc[current_sfc < terminus]))])

We then calculate the ablation as a function of the altitude:

In [ ]:
# Get the glacier top
top = current_sfc[0]

# By construct the ela ablation should be equal to the ela accumulation
ela_ablation = ela_accumulation

# Now calculate the annual ablation as a function of the altitude. Scaled by
# the width of the glacier.
ablation = (- ela_ablation + mb_grad *
            (current_sfc[current_sfc >= terminus] - ela_sfc) *
            1e-3 * mwidths[current_sfc >= terminus])

# Append 0 ablation downstream of glacier terminus
ablation = np.hstack([ablation,
                      np.zeros(len(current_sfc[current_sfc < terminus]))])

# Correct ablation > 0 to 0
ablation[ablation > 0] = 0

``ela_accumulation`` and ``ela_ablation`` are the accumulation and the ablation at the ELA, respectively - by construction, they should be equal:

In [ ]:
print(f'Mass balance at the ELA: {float(ela_accumulation - ela_ablation):.2f} m w.e. yr^-1')

Now, we can define the glacier surface after accumulation and ablation, respectively, and plot them on the glacier. The code below is mostly for plotting.

In [ ]:
# Accumulation and ablation surfaces
accumulation_sfc = current_sfc + accumulation
ablation_sfc = current_sfc + ablation

# net mass balance m w.e yr^-1
mb_we = accumulation + ablation

# Theoretical glacier surface without ice flow
# corrected to the bed where ice thickness is negative
mb_sfc = current_sfc + mb_we
# Where is there no ice?
no_ice = np.where(mb_sfc < bed_h)
# Correct it
mb_sfc[no_ice] = bed_h[no_ice] 

# plot the model glacier
plt.figure(figsize=(22,10))
ax = plt.subplot(121)
# Plot the bed
ax.plot(distance_along_glacier, bed_h, '--k', label='Bedrock')
# And the current glacier surface
ax.plot(distance_along_glacier, current_sfc, '--r',
        label='Current glacier surface')
# Fill the ice which is not affected by the accumulation
ax.fill_between(distance_along_glacier, bed_h, mb_sfc, mb_sfc <= current_sfc,
                color='grey', alpha=0.3)
# and ablation.
ax.fill_between(distance_along_glacier, bed_h, current_sfc,
                mb_sfc >= current_sfc, color='grey', alpha=0.3)

# Fill in the accumulation.
ax.fill_between(distance_along_glacier, current_sfc, mb_sfc,
                mb_sfc > current_sfc, color='deepskyblue', alpha=0.7,
                label='Net accumulation')
# Fill in the ablation.
ax.fill_between(distance_along_glacier, current_sfc, mb_sfc,
                mb_sfc <= current_sfc, color='red', alpha=0.3,
                label='Net ablation')
# Add the ela
ax.axhline(ela, ls='--', c='grey')
ax.text(distance_along_glacier[-1], ela+10, 'Initial ELA',
        horizontalalignment='right', verticalalignment='bottom',
        color='grey')

ax.set_xlabel('Distance along flowline (km)', fontsize=18)
ax.set_ylabel('Elevation (m)', fontsize=18)
plt.legend()

ax = plt.subplot(122)
plot_glacier_graphics('05')
ax = plt.gca()
ax.patch.set_visible(False)
ax.axis('off');
plt.tight_layout()

Although our initial glacier is in equilibrium, it doesn't mean that the mass-balance is zero everywhere. As visualised above, the net positive mass-balance at the top (more accumulation, less ablation) balances out the net negative mass balance at the tongue (less accumulation, more ablation).

As explained in the [accumulation and ablation notebook](accumulation_and_ablation.ipynb), the net accumulation at the top implies a flow of ice trough the glacier, which compensates for the melt in the lower areas. 

**At equilibrium, a glacier's net mass-balance is zero but ice is still moving from top to bottom.**

Now we have set the scene to model glacier advance and retreat.

## Advancing Glacier

To simulate a glacier advance, we will use the same glacier as before, but move the ELA down the glacier. This is similar to e.g. an abrupt climate cooling over the glacier:

In [ ]:
# Number of vertical grid points the ELA is shifted down the glacier
ela_shift_down = 10

# Define the ELA, this moves the down from the initial ela.
ela_adv = bed_h[np.where(widths == ELAW)[0][ela_shift_down]]

print(f'We move the ELA from initially {ela:.0f} m downward to {ela_adv:.0f} m')

With the new ELA we can define a new mass balance:

In [ ]:
mb_model_adv = LinearMassBalance(ela_adv, grad=mb_grad)

Then we initialise a new model, based on the current glacier geometry but with a new mass balance model:

In [ ]:
model_adv = FlowlineModel(model.fls[0], mb_model=mb_model_adv, y0=0,
                          min_dt=0, cfl_number=0.01)
# And run it to a new equilibrium
model_adv.run_until_equilibrium()

In [ ]:
# get the equilibrium surface height of our test glacier in equilibrium
current_sfc_adv = model_adv.fls[0].surface_h

Let's take a look at the new equilibrium state after the decreased ELA

In [ ]:
# plot the model glacier
plt.figure(figsize=(22,10))
# fig, ax = plt.subplots(1, 1, figsize=(16, 9))
ax = plt.subplot(121)
# Plot the bed
ax.plot(distance_along_glacier, bed_h, '--k', label='Bedrock')
# And the pre advance glacier surface
ax.plot(distance_along_glacier, current_sfc, '--r',
        label='Glacier surface before advance')

# And fill it in
ax.fill_between(distance_along_glacier, bed_h, current_sfc,
                color='grey', alpha=0.3)
# Plot the advancing glacier.
ax.plot(distance_along_glacier, current_sfc_adv, color='deepskyblue',
        label='Glacier surface after advance')
# And fill it in
ax.fill_between(distance_along_glacier, current_sfc, current_sfc_adv,
                current_sfc_adv >= current_sfc, color='deepskyblue',
                alpha=0.7)

# Add the old ela
ax.axhline(ela, ls='--', c='grey')
ax.text(distance_along_glacier[-1], ela+10, 'Initial ELA',
        horizontalalignment='right', verticalalignment='bottom',
        color='grey')

# Add the new ela
ax.axhline(ela_adv, ls='--', c='deepskyblue')
ax.text(distance_along_glacier[-1], ela_adv+10,
        'New ELA, e.g. colder climate',
        horizontalalignment='right', verticalalignment='bottom',
        color='deepskyblue')

ax.set_xlabel('Distance along flowline (km)', fontsize=18)
ax.set_ylabel('Elevation (m)', fontsize=18)
ax.set_title('Advancing glacier')
plt.legend()

ax = plt.subplot(122)
plot_glacier_graphics('09')
ax = plt.gca()
ax.patch.set_visible(False)
ax.axis('off');
plt.tight_layout()

Decreasing the ELA results in an increased accumulation area and thus an higher  accumulation. More mass at the top of the glacier will increase the ice flow $\vec{q}$ down the glacier, leading to an advance of the glacier terminus and thus expanding the ablation area. This is illustrated by the thick arrow in the plot to the right.

## Retreating glacier

Similarly we can simulate glacier retreat. We will begin with our first glacier, but this time move the ELA up the glacier. This is what would happen at for instance an abrupt warming of the climate.

In [ ]:
# Number of vertical grid points the ELA is shifted up the glacier.
# If you want to see the differences better you can increase here the
# value to e.g. 30
ela_shift_up = 10

# Define the ELA, this moves the down from the initial ela.
ela_rtr = bed_h[np.where(widths == acc_width[-ela_shift_up])[0][0]]

print(f'We move the ELA from initially {ela:.0f} m up to {ela_rtr:.0f} m')

With the new ELA we can define a new mass balance:

In [ ]:
mb_model_rtr = LinearMassBalance(ela_rtr, grad=mb_grad)

Then we initialise a new model, based on the current glacier geometry of the initial glacier but with a new mass balance model:

In [ ]:
model_rtr = FlowlineModel(model.fls[0], mb_model=mb_model_rtr, y0=0,
                          min_dt=0, cfl_number=0.01)
# And run it to a new equilibrium
model_rtr.run_until_equilibrium()

Now, we can calculate linear accumulation, ablation and mass balance profiles, for the retreating glacier:

In [ ]:
# get the equilibrium surface height of our test glacier in equilibrium
current_sfc_rtr = model_rtr.fls[0].surface_h

Plot the new equilibrium state

In [ ]:
# plot the model glacier
plt.figure(figsize=(22,10))
# fig, ax = plt.subplots(1, 1, figsize=(16, 9))
ax = plt.subplot(121)
# Plot the bed
ax.plot(distance_along_glacier, bed_h, '--k', label='Bedrock')
# And the pre advance glacier surface
ax.plot(distance_along_glacier, current_sfc, 
        label='Glacier surface before retreat')

# And fill it in
ax.fill_between(distance_along_glacier, bed_h, current_sfc,
                color='grey', alpha=0.3)
# Plot the advancing glacier.
ax.plot(distance_along_glacier, current_sfc_rtr, color='red',
        label='Glacier surface after retreat')
# And fill it in
ax.fill_between(distance_along_glacier, current_sfc, current_sfc_rtr,
                current_sfc_adv >= current_sfc, color='red',
                alpha=0.3)

# Add the old ela
ax.axhline(ela, ls='--', c='grey')
ax.text(distance_along_glacier[-1], ela+10, 'Initial ELA',
        horizontalalignment='right', verticalalignment='bottom',
        color='grey')

# Add the new ela
ax.axhline(ela_rtr, ls='--', c='red')
ax.text(distance_along_glacier[-1], ela_rtr+10,
        'New ELA, e.g. warmer climate',
        horizontalalignment='right', verticalalignment='bottom',
        color='red')

ax.set_xlabel('Distance along flowline (km)', fontsize=18)
ax.set_ylabel('Elevation (m)', fontsize=18)
plt.legend()

ax = plt.subplot(122)
plot_glacier_graphics('11')
ax = plt.gca()
ax.patch.set_visible(False)
ax.axis('off');
plt.tight_layout()

The increase in the ablation area leads to a net mass loss compared to the former glacier extent. When our glacier with a higher ELA is in equilibrium, the glacier has retreated: 

Raising the ELA results in an decrease in the accumulation area and accumulation while the ablation area increases. Less accumulation also leads to an decreased ice flow $\vec{q}$ down the glacier, which combined with the increased accumulation results in a retreat of the glacier terminus.

## Take home points

- The equilibrium line altitude (ELA) is the altitude on a glacier where accumulation and ablation are in balance, $\dot{m}(z) = 0$ at $z=$ ELA
- A decrease in the ELA leads to:
    1. Increased accumulation
    2. An initial decrease of the ablation area
    3. A net mass gain resulting in an increased ice flux down the glacier
    4. Glacier advance
- An increase in the ELA leads to:
    1. A decreased accumulation area
    2. An increased ablation area
    3. A net mass loss resulting in a decreased ice flux down the glacier
    4. Glacier retreat

## What's next?

[Back to the table of contents](../welcome.ipynb)